In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import xarray as xr
import intake

import config
import oco2mip_ff
import util

In [4]:
use_dask = False

if use_dask:
    cluster, client = util.get_ClusterClient()
    cluster.scale(12) #adapt(minimum_jobs=2, maximum_jobs=5)
    cluster

In [6]:
version = 'v2020.1'
yr1, yr2 = 2000, 2020

date_range = pd.date_range(start=f'{yr1:04d}-01-01', end=f'{yr2:04d}-12-31', freq='1M')

if use_dask:
    ds_list_list = client.map(oco2mip_ff.retrieve_datasets, date_range, version=version)
    ds_list_list = client.gather(ds_list_list)
else:
    ds_list_list = []
    for date in date_range:
        ds_list_list.append(oco2mip_ff.retrieve_datasets(date, version=version))
        
ds_list = [ds for ds_list in ds_list_list for ds in ds_list]

ds = xr.concat([ds[['SFCO2_FF', 'time_bnds']] for ds in ds_list], dim='time', compat='override', coords='minimal').squeeze()
ds['area'] = ds_list[0].area
for v in ds.variables:
    ds[v].encoding = ds_list[0][v].encoding

ds

<xarray.Dataset>
Dimensions:    (time: 7671, lat: 180, lon: 360, d2: 2)
Coordinates:
  * lon        (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 177.5 178.5 179.5
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time       (time) object 2000-01-01 12:00:00 ... 2020-12-31 12:00:00
Dimensions without coordinates: d2
Data variables:
    SFCO2_FF   (time, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    time_bnds  (time, d2) object 2000-01-01 00:00:00 ... 2021-01-01 00:00:00
    area       (lat, lon) float64 dask.array<chunksize=(180, 360), meta=np.ndarray>

In [7]:
ds.SFCO2_FF.weighted(ds.area).sum(['lat', 'lon'])

<xarray.DataArray 'SFCO2_FF' (time: 7671)>
dask.array<sum-aggregate, shape=(7671,), dtype=float64, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2000-01-01 12:00:00 ... 2020-12-31 12:00:00

In [8]:
file_out = f"{config.flux_product_dir}/SFCO2_FF.OCO2-MIP-FF.{version}.{yr1:04d}0101-{yr2:04d}1231.nc"
ds.attrs['description'] = f'daily integral of Fossil Fuel CO2 Emissions for the OCO2 Model Intercomparison Project (MIP)'
ds.attrs['source'] = 'https://zenodo.org/record/4776925'
ds.attrs['authors'] = 'Basu, Sourish;  Nassar, Ray'
ds.attrs['note'] = 'daily integration and compilation by Matt Long (NCAR)'
util.to_netcdf_clean(ds, file_out)

------------------------------
Writing /glade/work/mclong/sno-analysis/flux-products/SFCO2_FF.OCO2-MIP-FF.OCO2-MIP.20000101-20201231.nc
netcdf SFCO2_FF.OCO2-MIP-FF.OCO2-MIP.20000101-20201231 {
dimensions:
	time = 7671 ;
	lat = 180 ;
	lon = 360 ;
	d2 = 2 ;
variables:
	float SFCO2_FF(time, lat, lon) ;
		SFCO2_FF:_FillValue = 9.96921e+36f ;
		SFCO2_FF:comment = "ODIAC fossil fuel flux convolved with TIMES emission factors" ;
		SFCO2_FF:authors = "ODIAC from Tomohiro Oda (tom.oda@nasa.gov), TIMES from Ray Nassar (ray.nassar@canada.ca), convolved by Sourish Basu (sourish.basu@noaa.gov)" ;
		SFCO2_FF:long_name = "Fossil fuel flux" ;
		SFCO2_FF:units = "mol/m^2/s" ;
		SFCO2_FF:cell_methods = "time: sum" ;
	double time_bnds(time, d2) ;
		time_bnds:long_name = "Bounds for time integration" ;
	double lon(lon) ;
		lon:units = "degrees_east" ;
		lon:long_name = "Longitude" ;
	double lat(lat) ;
		lat:units = "degrees_north" ;
		lat:long_name = "Latitude" ;
	double time(time) ;
		time:bounds = "time

In [9]:
curator = util.curate_flux_products()
curator.add_source(
        key=f"SFCO2_FF.OCO2-MIP.{version}",
        urlpath=file_out,
        description='daily integral of Fossil Fuel CO₂ Emissions for the OCO2 Model Intercomparison Project (MIP)',
    )

In [10]:
if use_dask:
    del client
    del cluster